# Bokeh

Bokeh es una biblioteca de visualización interactiva de Python que se dirige a navegadores web modernos para su presentación. Su objetivo es proporcionar una construcción elegante y concisa de gráficos novedosos al estilo de D3.js y ampliar esta capacidad con interactividad de alto rendimiento en conjuntos de datos de gran tamaño o de streamming. Bokeh puede ayudar a cualquiera que desee crear trazados interactivos, cuadros de mando y aplicaciones de datos de manera rápida y fácil.

## Ejemplo simple
Aquí hay un primer ejemplo simple. Primero importaremos la función de figura de `bokeh.plotting`, lo que nos permitirá crear todo tipo de tramas interesantes fácilmente. También importamos las funciones `show()` y `ouptut_notebook()`  de  `bokeh.io`; estas nos permiten mostrar nuestros resultados en línea en la notebook.

In [ ]:
#!pip install bokeh

In [1]:
from bokeh.plotting import figure 
from bokeh.io import output_notebook, show
import pandas as pd
import numpy as np

A continuación, le diremos a Bokeh que muestre sus gráficos directamente en la notebook ejecutando `output_notebook()`. Esto hará que todo el JavaScript y los datos sean insertados directamente en el HTML de la notebook. Alternativamente, Bokeh también puede mostrar directamente en archivos HTML o usar un servidor, como veremos más adelante.

In [2]:
output_notebook()

Loading BokehJS ...

Importamos NumPy y generamos algunos datos

In [3]:
from numpy import cos, linspace
x = np.linspace(-15, 15, 200)
y = x*cos(x)

Ahora llamaremos a la función `figure` de Bokeh para crear un plot  `p`. Luego llamamos al método  `circle()` del plot para representar un círculo rojo en cada uno de los puntos en x e y.

Podemos interactuar de inmediato con el plot

*haga clic y arrastre alrededor de la trama.
*la rueda del mouse se acercará y alejará (después de habilitarla en la barra de herramientas)

La siguiente barra de herramientas es la predeterminada que está disponible para todos los gráficos. Se puede configurar más a través del keyword argument `tools`





In [4]:
p = figure(width=500, height=500)
p.circle(x, y, size=7, color="firebrick", alpha=0.5)
show(p)

## Bar Plot 

El modelo de display de Bokeh se basa en la composición de primitivas gráficas que están ligadas a series de datos. Esto es similar en espíritu a Protovis y D3, y diferente de la mayoría de las otras bibliotecas de trazado de Python.

Un ejemplo un poco más sofisticado demuestra esta idea.

Bokeh se importa con un pequeño conjunto de "datos de muestra" interesantes en el paquete  `bokeh.sampledata` . Cargaremos algunos datos históricos de kilometraje del automóvil.

In [5]:
from bokeh.sampledata.autompg import autompg
autompg.sample(5)

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
144,28.0,4,90.0,75,2125,14.5,74,1,dodge colt
172,18.0,6,171.0,97,2984,14.5,75,1,ford pinto
382,26.0,4,156.0,92,2585,14.5,82,1,chrysler lebaron medallion
215,30.0,4,111.0,80,2155,14.8,77,1,buick opel isuzu deluxe
286,18.2,8,318.0,135,3830,15.2,79,1,dodge st. regis


In [6]:
df_mpg_group = autompg.groupby("yr")["mpg"].agg(['mean', 'std'])
df_mpg_group.head()

,mean,std
yr,,
70,17.689655,5.339231
71,21.111111,6.675635
72,18.714286,5.435529
73,17.100000,4.700245
74,22.769231,6.537937


In [7]:
df_mpg_group.reset_index(inplace = True)
df_mpg_group

,yr,mean,std
0,70,17.689655,5.339231
1,71,21.111111,6.675635
2,72,18.714286,5.435529
3,73,17.100000,4.700245
4,74,22.769231,6.537937
5,75,20.266667,4.940566
6,76,21.573529,5.889297
7,77,23.375000,6.675862
8,78,24.061111,6.898044
9,79,25.093103,6.794217



Para cada año, queremos trazar la distribución de MPG dentro de ese año.

In [8]:
p = figure(title="MPG Por Año (Japón y USA)")

p.vbar(x      = df_mpg_group['yr'],
       bottom = df_mpg_group['mean'] - df_mpg_group['std'],
       top    = df_mpg_group['mean'] + df_mpg_group['std'],
       width=0.8,
       fill_alpha=0.2,
       line_color=None, legend="MPG 1 desvío")

p.circle(x = autompg.loc[autompg.origin==3, "yr"],
         y = autompg.loc[autompg.origin==3, "mpg"],
         size=10,
         alpha=0.5,
         color="red",
         legend="Japanese")

p.triangle(x = autompg.loc[autompg.origin==1, "yr"],
           y = autompg.loc[autompg.origin==1, "mpg"],
           size=10,
           alpha=0.3,
           color="blue",
           legend="American")

p.legend.location = "top_left"
p.title.text_font_size = '16pt'
p.title.text_color = 'blue'

show(p)

## Linked Brushing
Para vincular plots al nivel  de datos, podemos envolver de manera explícita los datos en un 
`ColumnDataSource`. Esto nos permite hacer referencia a columnas por nombre.

Podemos usar una herramienta de "selección" para seleccionar puntos en un plot, y los puntos vinculados en los otros plots resaltarán.

In [9]:
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

data_bokeh = ColumnDataSource(autompg)

options = {'plot_width': 300,
           'plot_height': 300,
           'tools': 'pan, wheel_zoom, box_zoom, box_select, lasso_select, reset'}

p1 = figure(title="MPG por Año", x_axis_label = "Año", y_axis_label = "MPG", **options)
p1.circle("yr", "mpg", color = "blue", source = data_bokeh)

p2 = figure(title="HP vs. Desplazamiento", x_axis_label = "HP", y_axis_label = "Desplazamiento", **options)
p2.cross("hp", "displ", color = "green", source = data_bokeh)

p3 = figure(title="MPG vs. Desplazamiento", x_axis_label = "MPG", y_axis_label = "Desplazamiento", **options)
p3.diamond("mpg", "displ", size = "cyl", line_color="red", fill_color = None, source = data_bokeh)

p4 = figure(title="Aceleración vs. Desplazamiento", x_axis_label = "Aceleración", y_axis_label = "Desplazamiento",  **options)
p4.triangle("accel", "displ", color = "yellow", source = data_bokeh)

p = gridplot([[p1, p2], [p3, p4]] , toolbar_location="right")

show(p)

## Standalone HTML

Además de funcionar bien en la notebook, Bokeh también puede guardar plots en sus propios archivos HTML. Aquí está el ejemplo del diagrama de barra desde arriba, pero guardado en su propio archivo independiente. En esta ocasión, en lugar de utilizar `output_notebook()`, usaremos `output_file()`

Ahora, cuando llamamos a `show()`, también se abre una nueva pestaña del navegador con el gráfico. Si sólo quisiéramos guardar el archivo, usaríamos `save()` en su lugar.

In [10]:
from bokeh.plotting import output_file, save

output_file("barplot.html")
p = figure(title="MPG Por Año (Japón y USA)")

p.vbar(x      = df_mpg_group['yr'],
       bottom = df_mpg_group['mean'] - df_mpg_group['std'],
       top    = df_mpg_group['mean'] + df_mpg_group['std'],
       width=0.8,
       fill_alpha=0.2,
       line_color=None, legend="MPG 1 desvío")

p.circle(x = autompg.loc[autompg.origin==1, "yr"],
         y = autompg.loc[autompg.origin==1, "mpg"],
         size=10,
         alpha=0.5,
         color="red",
         legend="Japanese")

p.triangle(x = autompg.loc[autompg.origin==2, "yr"],
           y = autompg.loc[autompg.origin==2, "mpg"],
           size=10,
           alpha=0.3,
           color="blue",
           legend="American")

p.legend.location = "top_left"

show(p)

## Bokeh Applications

Bokeh también tiene un componente de [servidor](https://bokeh.pydata.org/en/latest/docs/user_guide/server.html#userguide-server) que se puede usar para crear aplicaciones web interactivas que conectan fácilmente la poderosa constelación de herramientas PyData con [sofisticadas visualizaciones de Bokeh](https://bokeh.pydata.org/en/latest/docs/gallery.html#gallery-server-examples).